In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
from collections import Counter
torch.__version__

'2.8.0'

# 5.2 Pytorch处理结构化数据

## 简介
结构化数据，即高度组织和整齐格式化的数据。它是可以放入表格和电子表格中的数据类型。

对于我们来说，结构化数据就是一张2维的表格，例如一个csv文件，就是结构化数据，在英文中一般称作Tabular Data。

## 数据预处理
我们拿到的结构化数据，一般都是一个csv文件或者数据库中的一张表格，所以对于结构化的数据，我们直接使用`pandas`库处理就可以了。

In [ ]:
df = pd.read_csv('./data/adult.csv')
df['salary'].unique()  # 查看salary列有哪些取值


In [ ]:
# 查看数据类型
df.head()

In [ ]:
df.describe()

In [ ]:
# 查看一共有多少数据
len(df)

对于模型的训练，只能够处理数字类型的数据，所以这里面我们首先要将数据分成三个类别：

- 训练的结果标签：即训练的结果，通过这个结果我们就能够明确地知道我们这次训练的任务是什么，是分类的任务，还是回归的任务。
- 分类数据：这类数据是离散的，无法通过直接输入在模型中进行训练，所以我们在浴池里的时候需要有限对这部分数据进行处理，这也是数据预处理的主要工作之一。
- 数值型数据：这类数据是可以直接输入到模型中的，但是这部分数据有可能还是离散的，所以如果需要也可以对其进行处理，并且处理后会对训练的精度有很大的提升。

In [ ]:
# 训练结果
result_var = 'salary'

# 分类型数据
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# 数值型数据
cont_names = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']




人工确认完数据类型后，我们可以看一下分类类型的数量和分布情况

In [ ]:
for col in df.columns:
    if col in cat_names:
        ccol = Counter(df[col])
        print(col, len(ccol), ccol)
        print("\r\n")

下一步就是要将分类型数据转成数字型数据，在这部分里面，我们还做了对于缺失数据的填充

In [ ]:
for col in df.columns:
    if col in cat_names:
        df[col].fillna('---')
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))
    
    if col in cont_names:
        df[col] = df[col].fillna(0)

上面的代码中：

我们首先使用了`pandas`的`fillna`函数对分类的数据进行了空值的填充，这里面标识成一个与其他现有值不一样的值就可以。在此使用`---`进行标记。然后就是使用了sklearn的`labelEncoder`函数进行了数据的处理。

然后对我们的数值型的数据进行了0填充的处理，对于数值型数据的填充，也可以使用平均值，或者其他方式填充。

In [ ]:
df.head()

数据处理完成后可以看到，现在所有的数据都是数字类型的了，可以直接输入到模型中进行训练了。

In [ ]:
Y = df['salary']
Y_label = LabelEncoder()
Y = Y_label.fit_transform(Y)
Y

In [ ]:
X = df.drop(columns = result_var)
X

## 定义数据集
要使用`pytorch`处理数据，肯定要使用`Dataset`进行数据集的定义

In [ ]:
class tabularDataset(Dataset):
    def __init__(self, X, Y):
        self.x = X.values
        self.y = Y
    
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

In [ ]:
train_ds = tabularDataset(X, Y)

In [ ]:
train_ds[0]

## 定义模型


In [ ]:
class tabularModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(14, 500)
        self.lin2 = nn.Linear(500, 100)
        self.lin3 = nn.Linear(100, 2)
        self.bn_in = nn.BatchNorm1d(14)
        self.bn1 = nn.BatchNorm1d(500)
        self.bn2 = nn.BatchNorm1d(100)

    def forward(self, x_in):
        x = self.bn_in(x_in)
        x = F.relu(self.lin1(x))
        x = self.bn1(x)

        x = F.relu(self.lin2(x))
        x = self.bn2(x)

        x = self.lin3(x)
        x = torch.sigmoid(x)
        return x



## 训练

In [ ]:
DEVICE = torch.device("cpu")

if torch.cuda.is_avaliable():
    DEVICE = torch.device("cuda")

print(DEVICE)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
model = tabularModel().to(DEVICE)
print(model)

In [ ]:
rn = torch.rand(3, 14).to(DEVICE)
model(rn)